In [ ]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os

In [15]:
#Lấy all data trong folder
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    try:    
        try:
            return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
        except:
            return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%Y/%m/%d %H:%M:%S").strftime(my_format)
def input_data( data_dir  ):
    list_files = []
        
    for filename in pathlib.Path(data_dir).glob('**/*.csv'):
        df = pd.read_csv(filename, dtype={'object_id':object})
        df['1_resolve_time'] = df['1_resolve_time'].apply(convert_datetime)
        df['1_resolve_time'] = pd.to_datetime(df['1_resolve_time'])
        list_files.append(df)

    df1 = pd.concat(list_files, ignore_index=True)
    return df1

In [16]:
#Lấy data (cần đổi source)
EUPL_R1_Rawdata = input_data('D:/UPL/EUPL/R1')
EUPL_R2_Rawdata = input_data('D:/UPL/EUPL/R2')

GUPL_R1_Rawdata = input_data('D:/UPL/GUPL/R1')
GUPL_R2_Rawdata = input_data('D:/UPL/GUPL/R2')

In [17]:
#Đưa title R2 giống R1 để so sánh điều kiện 2
EUPL_R2_Rawdata['title_2'] = EUPL_R2_Rawdata['title'].str.replace(' 2nd-Round','')
GUPL_R2_Rawdata['title_2'] = GUPL_R2_Rawdata['title'].str.replace(' 2nd-Round','')

In [18]:
#Add thêm colum Result
EUPL_R1_Rawdata.loc[EUPL_R1_Rawdata['1_annotation'].isnull(),'1st result']="Approve"
EUPL_R1_Rawdata.loc[EUPL_R1_Rawdata['1st result'].isnull(),'1st result']="Disapprove"
EUPL_R2_Rawdata.loc[EUPL_R2_Rawdata['1_annotation'].isnull(),'2nd result']="Approve"
EUPL_R2_Rawdata.loc[EUPL_R2_Rawdata['2nd result'].isnull(),'2nd result']="Disapprove"

GUPL_R1_Rawdata.loc[GUPL_R1_Rawdata['1_annotation'].isnull(),'1st result']="Approve"
GUPL_R1_Rawdata.loc[GUPL_R1_Rawdata['1st result'].isnull(),'1st result']="Disapprove"
GUPL_R2_Rawdata.loc[GUPL_R2_Rawdata['1_annotation'].isnull(),'2nd result']="Approve"
GUPL_R2_Rawdata.loc[GUPL_R2_Rawdata['2nd result'].isnull(),'2nd result']="Disapprove"

In [19]:
#Merge
EUPL_Merge_File = pd.merge(EUPL_R1_Rawdata,EUPL_R2_Rawdata,how = 'inner', on= 'object_id')
GUPL_Merge_File = pd.merge(GUPL_R1_Rawdata,GUPL_R2_Rawdata,how = 'inner', on= 'object_id')

In [20]:
# Rename Column

EUPL_Merge_File.rename(columns = {'title_x':'1st Queue Name',
                                  'task_id_x':'1st Task ID',
                                  'object_id':'Object ID',
                                  '1_verifier_x':'1st Name',
                                  '1_resolve_time_x':'1st Date',
                                  '1_annotation_x':'1st Rejection Reason',
                                  '1_duration_x':'1st AHT',
                                  
                                  'task_id_y':'2nd Task ID',
                                  'title_y':'2nd Queue Name',
                                  'project_id_y':'Queue ID',
                                  '1_verifier_y':'2nd Name',
                                  '1_annotation_y':'2nd Rejection Reason',
                                  'tcs_link_y':'TCS Link',
                                  '1_resolve_time_y':'2nd Date'
                     },inplace=True)

GUPL_Merge_File.rename(columns = {'title_x':'1st Queue Name',
                                  'task_id_x':'1st Task ID',
                                  'object_id':'Object ID',
                                  '1_verifier_x':'1st Name',
                                  '1_resolve_time_x':'1st Date',
                                  '1_annotation_x':'1st Rejection Reason',
                                  '1_duration_x':'1st AHT',
                                  
                                  'task_id_y':'2nd Task ID',
                                  'title_y':'2nd Queue Name',
                                  'project_id_y':'Queue ID',
                                  '1_verifier_y':'2nd Name',
                                  '1_annotation_y':'2nd Rejection Reason',
                                  'tcs_link_y':'TCS Link',
                                  '1_resolve_time_y':'2nd Date'
                     },inplace=True)

In [21]:
EUPL_Merge_File[['Queue ID','Object ID','1st Task ID','2nd Task ID']] = EUPL_Merge_File[['Queue ID','Object ID','1st Task ID','2nd Task ID']].replace(to_replace=r'id=',value= '',regex=True).astype(str)
GUPL_Merge_File[['Queue ID','Object ID','1st Task ID','2nd Task ID']] = GUPL_Merge_File[['Queue ID','Object ID','1st Task ID','2nd Task ID']].replace(to_replace=r'id=',value= '',regex=True).astype(str)

In [22]:
EUPL_Merge_File.loc[EUPL_Merge_File['1st Date'] < EUPL_Merge_File['2nd Date'],"Check time"]="Yes"
GUPL_Merge_File.loc[GUPL_Merge_File['1st Date'] < GUPL_Merge_File['2nd Date'],"Check time"]="Yes"

EUPL_Merge_File.loc[EUPL_Merge_File['title_2'] == EUPL_Merge_File['1st Queue Name'],"Check queue"]="Yes"
GUPL_Merge_File.loc[GUPL_Merge_File['title_2'] == GUPL_Merge_File['1st Queue Name'],"Check queue"]="Yes"

In [23]:
#Filter các giá trị thỏa mãn cả 2 điều kiện trên
EUPL_Merge_File = EUPL_Merge_File[EUPL_Merge_File['Check time']=="Yes"]
EUPL_Merge_File = EUPL_Merge_File[EUPL_Merge_File['Check queue']=="Yes"]

#Sắp xếp và lấy những column cần thiết
EUPL_Merge_File = EUPL_Merge_File[['2nd Queue Name','Queue ID','Object ID','1st Name','1st Date','1st Task ID','1st result','1st Rejection Reason','1st AHT','2nd Name','2nd Date','2nd Task ID','TCS Link','2nd result','2nd Rejection Reason']]
GUPL_Merge_File['2nd Queue Name'].value_counts() #2check data với TCS output

# # Lọc các giá trị trong khoảng thời gian từ '2023-06-26' đến '2023-07-02' trên cột '2nd Date'
# EUPL_Merge_File_filtered = EUPL_Merge_File.loc[(EUPL_Merge_File['2nd Date'] >= '2023-06-26') & (EUPL_Merge_File['2nd Date'] <= '2023-07-02')]
# EUPL_Merge_File_filtered

2nd Queue Name
[VN] Product Sample Evaluate 2nd-Round         13456
[VN] merge video Sampling Evaluat 2nd-Round     3777
[VN]livestream Sampling Evaluate 2nd-Round      3452
[VN] CB Product Sample Evaluation 2nd-Round      316
Name: count, dtype: int64

In [24]:
#Filter các giá trị thỏa mãn cả 2 điều kiện trên
GUPL_Merge_File = GUPL_Merge_File[ GUPL_Merge_File['Check time']=="Yes"]
GUPL_Merge_File = GUPL_Merge_File[GUPL_Merge_File['Check queue']=="Yes"]

#Sắp xếp và lấy những column cần thiết
GUPL_Merge_File = GUPL_Merge_File[['2nd Queue Name','Queue ID','Object ID','1st Name','1st Date','1st Task ID','1st result','1st Rejection Reason','1st AHT','2nd Name','2nd Date','2nd Task ID','TCS Link','2nd result','2nd Rejection Reason']]
GUPL_Merge_File['2nd Queue Name'].value_counts() #2check data với TCS output

2nd Queue Name
[VN] Product Sample Evaluate 2nd-Round         10931
[VN]livestream Sampling Evaluate 2nd-Round      3452
[VN] merge video Sampling Evaluat 2nd-Round     3421
[VN] CB Product Sample Evaluation 2nd-Round      227
Name: count, dtype: int64

In [25]:
#filter time E-UPL
EUPL_Merge_File['Mod_Week'] = EUPL_Merge_File['1st Date'].apply(lambda x: x.isocalendar()[1])
EUPL_Merge_File['QA_Week'] = EUPL_Merge_File['2nd Date'].apply(lambda x: x.isocalendar()[1])
EUPL_Merge_File.loc[EUPL_Merge_File['Mod_Week'] == EUPL_Merge_File['QA_Week'],"Check week"]="Yes"
EUPL_Merge_File = EUPL_Merge_File[EUPL_Merge_File['Check week']=="Yes"]

abc ="_"
def filter_time( df,  from_date, to_date):

    UPL_rawdata = df[(df['2nd Date'].dt.strftime('%Y-%m-%d') >= from_date) & (df['2nd Date'].dt.strftime('%Y-%m-%d') <= to_date) ]
    UPL_rawdata = UPL_rawdata[['2nd Queue Name','Queue ID','Object ID','1st Name','1st Date','1st Task ID','1st result','1st Rejection Reason','1st AHT','2nd Name','2nd Date','2nd Task ID','TCS Link','2nd result','2nd Rejection Reason']]   
    UPL_rawdata.to_excel(f"C:/Users/v6210227/Desktop/E-UPL {from_date}{abc}{to_date}.xlsx", index=False)

    return UPL_rawdata

EUPL_Merged = filter_time(EUPL_Merge_File, '2023-07-24','2023-07-30')
#Nhập lại ngày tháng muốn export data

In [26]:
#filter time G-UPL
GUPL_Merge_File['Mod_Week'] = GUPL_Merge_File['1st Date'].apply(lambda x: x.isocalendar()[1])
GUPL_Merge_File['QA_Week'] = GUPL_Merge_File['2nd Date'].apply(lambda x: x.isocalendar()[1])
GUPL_Merge_File.loc[GUPL_Merge_File['Mod_Week'] == GUPL_Merge_File['QA_Week'],"Check week"]="Yes"
GUPL_Merge_File = GUPL_Merge_File[GUPL_Merge_File['Check week']=="Yes"]

abc ="_"
def filter_time( df,  from_date, to_date):

    UPL_rawdata = df[(df['2nd Date'].dt.strftime('%Y-%m-%d') >= from_date) & (df['2nd Date'].dt.strftime('%Y-%m-%d') <= to_date) ]
    UPL_rawdata = UPL_rawdata[['2nd Queue Name','Queue ID','Object ID','1st Name','1st Date','1st Task ID','1st result','1st Rejection Reason','1st AHT','2nd Name','2nd Date','2nd Task ID','TCS Link','2nd result','2nd Rejection Reason']]
    UPL_rawdata.to_excel(f"C:/Users/v6210227/Desktop/G-UPL {from_date}{abc}{to_date}.xlsx", index=False)

    return UPL_rawdata

GUPL_Merged = filter_time(GUPL_Merge_File, '2023-07-24','2023-07-30')
#Nhập lại ngày tháng muốn export data